# CS349 Final: Coronavirus Experiments

## Initialization

In [15]:
import os
from utils import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --------- HYPERPARAMETERS ------------
BASE_PATH = './COVID-19/csse_covid_19_data/'
TIME_SERIES_PATH = 'csse_covid_19_time_series'
US_CONFIRMED_PATH = 'time_series_covid19_confirmed_US.csv'
GLOBAL_CONFIRMED_PATH= 'time_series_covid19_confirmed_US.csv'
# --------------------------------------



## US State-by-State Analysis

### Load State Time Series Data

In [34]:

us_confirmed = os.path.join(BASE_PATH, TIME_SERIES_PATH, US_CONFIRMED_PATH)
us_confirmed = data.load_csv_data(us_confirmed)


# Aggregate data for each state
cases = []
labels = []

for state in np.unique(us_confirmed['Province_State']):
    df = data.filter_by_attribute(
        us_confirmed, "Province_State", state)
    state_cases = np.array(df.iloc[:, 11:])
    cases.append(list(np.sum(state_cases, axis=0)))
    labels.append(state)

cases = np.array(cases)
labels = np.array(labels)
    

### Visualize Data

[[1, 2, 3, 4], [4, 5, 6, 7]]
None
